In [11]:
import pandas as pd

# Load bank CSV
df = pd.read_csv("/home/sehar/INTELLIGENT-EXPENSE-TRACKER/data/raw/cibc_data.csv",header=None)
# Example columns: ['Date', 'Merchant', 'Description', 'Amount']

# Load Splitwise CSV
# df_splitwise = pd.read_csv("splitwise_expenses.csv")
# # Example columns: ['Date', 'Group', 'Note', 'Amount', 'PaidBy']
# Rename columns explicitly
df.columns = ["date", "merchant", "debit", "credit", "card_no"]

# Convert debit/credit to numeric
df["debit"] = pd.to_numeric(df["debit"], errors="coerce")
df["credit"] = pd.to_numeric(df["credit"], errors="coerce")

# Create signed amount column
df["amount"] = df["credit"].fillna(0) - df["debit"].fillna(0)

df["date"] = pd.to_datetime(df["date"])
df_clean = df[["date", "merchant", "amount"]]

df_clean.to_csv('/home/sehar/INTELLIGENT-EXPENSE-TRACKER/data/raw/first_batch.csv', index=False)

print(df_clean.head(10))


        date                                 merchant  amount
0 2025-09-04       PRE-AUTHORIZED PAYMENT - THANK YOU  391.07
1 2025-09-02       2574 GAP OUTLET CANADA LASALLE, ON  -27.11
2 2025-09-02     DD/DOORDASHINDIAPARADI VANCOUVER, BC  -16.14
3 2025-09-02               WALMART.CA MISSISSAUGA, ON  -69.36
4 2025-09-02                   CAW MARKET WINDSOR, ON   -1.92
5 2025-08-26              DOLLARAMA # 789 WINDSOR, ON  -17.52
6 2025-08-21               POPEYES #12213 WINDSOR, ON   -6.41
7 2025-08-21         UBER CANADA/UBEREATS TORONTO, ON  -11.81
8 2025-08-19  FIDO Mobile ******8845 888-481-3436, ON  -38.42
9 2025-08-19     TERANET POA HANDLING FEE TORONTO, ON   -3.00


In [38]:
import re

# Load the original CSV
df = pd.read_csv("/home/sehar/INTELLIGENT-EXPENSE-TRACKER/data/raw/second_batch.csv")

# Make a fresh copy of relevant columns
df_new = df[["date", "merchant", "amount"]].copy()

def clean_merchant(text):
    text = text.upper()
    text = re.sub(r"\d{4,}", "", text)      # remove long numbers
    text = re.sub(r"\s+[A-Z]{2}$", "", text) # remove province codes like "ON"
    text = re.sub(r"[^A-Z ]", "", text)      # keep only letters/spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

# If df_clean was a slice like df[df["amount"] > 0]
# df_clean = df_clean.copy()  # now it's a true independent DataFrame

# Now apply your cleaning safely
df_new["merchant_clean"] = df_new["merchant"].apply(clean_merchant)

# Define a mapping of keywords → categories
category_rules = {
    "WALMART": "Grocery",
    "OUTLET": "Shopping",
    "POPEYES": "Food",
    "UBER": "Food",
    "DOORDASH": "Food",
    "FIDO": "Utility",
    "MOBILE": "Utility",
    "DOLLAR": "Shopping",
    "CAW MARKET": "Grocery",
    "PAYMENT": "Bills",
    "FRESHCO": "Grocery",
    "TIM HORTONS": "Food",
    "AMAZON": "Shopping",
    "HM": "Shopping",
    "MCDONALDS": "Food",
    "SHOPPERS": "Shopping",
    "SK": "Grocery",
    "BANFF": "trip",
    "WINNIPEG": "trip",
    "CALGARY": "trip",
    "VICTORIA": "trip",
    "CASHBACK": "cashback",
    "ONROUTE": "trip",
    "RESTAURANT": "Food",
    "INDIA": "Food",
    "CHICKF": "Food",
    "INSTACART": "Grocery",
    "PIZZA": "Food",
    "PAAN": "Food",
    "AW": "Food",
    "LCBO": "Food",
    "WINNERS": "Shopping",
    "MARKET": "Food",
    "ARDEN": "Shopping",
    "CEI": "Food",
    "BASICS": "Grocery",
    "FALLS": "trip",
    "STARBUCKS": "Food",
    "PAAN": "Food",
    "SEPHORA": "Shopping"

    }

# Function to assign category based on merchant name
def categorize_merchant(merchant):
    for keyword, category in category_rules.items():
        if keyword in merchant:
            return category
    return "Other"  # default category

# Apply mapping
df_new["category"] = df_new["merchant_clean"].apply(categorize_merchant)

# Check results
print(df_new.head())


df_new.to_csv("/home/sehar/INTELLIGENT-EXPENSE-TRACKER/data/raw/fouth_batch.csv",index=False)

         date                         merchant  amount  \
0  2025-09-04  PREAUTHORIZED PAYMENT THANK YOU  391.07   
1  2025-09-02        GAP OUTLET CANADA LASALLE  -27.11   
2  2025-09-02  DDDOORDASHINDIAPARADI VANCOUVER  -16.14   
3  2025-09-02            WALMARTCA MISSISSAUGA  -69.36   
4  2025-09-02               CAW MARKET WINDSOR   -1.92   

                    merchant_clean  category  
0  PREAUTHORIZED PAYMENT THANK YOU     Bills  
1        GAP OUTLET CANADA LASALLE  Shopping  
2  DDDOORDASHINDIAPARADI VANCOUVER      Food  
3            WALMARTCA MISSISSAUGA   Grocery  
4               CAW MARKET WINDSOR   Grocery  


In [39]:
# Count how many transactions per category
category_counts = df_new["category"].value_counts()

print(category_counts)


category
Food        142
Shopping    132
Grocery      87
Other        83
Bills        29
trip         28
Utility      23
cashback      6
Name: count, dtype: int64
